<center><h2>Setup - Environment & Dependencies</center></h2>

In [1]:
#Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import modf
from scipy import stats

%matplotlib inline

plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.axisbelow'] = True

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Import cleaned up CSV files
surface = pd.read_csv('./Resources/Cleaned/ttc_surface_route_delays.csv')
weather = pd.read_csv('./Resources/Cleaned/weather.csv')
holidays = pd.read_csv('./Resources/Cleaned/holidays.csv')

In [3]:
#Merging Surface data, weather and holidaya
surface['Report Date'] = pd.to_datetime(surface['Report Date'])
surface['Month'] = surface['Report Date'].dt.month
surface['nDay'] = surface['Report Date'].dt.day
surface = surface.rename(columns={'Report Date': 'Date'})
surface = surface.drop(columns=['Unnamed: 0'])

weather['Date/Time'] = pd.to_datetime(weather['Date/Time'])
weather = weather.rename(columns={
    'Date/Time': 'Date', 
    'Mean Temp (°C)': 'Mean Temp', 
    'Total Precip (mm)': 'Precipitation'
})
weather = weather.drop(columns={'Year', 'Month', 'Day'})

holidays = holidays.rename(columns={'date': 'Date', 'holiday': 'Holiday'})
holidays['Date'] = pd.to_datetime(holidays['Date'])

surface = pd.merge(surface, weather, on ='Date', how='left')
surface = pd.merge(surface, holidays, on ='Date', how='left')

surface['Holiday'] = surface['Holiday'].fillna('None')

surface = surface[surface['Min Delay'] < 7000]
surface['Route'] = surface['Route'].astype('category')
surface['Vehicle'] = surface['Vehicle'].astype('category')

surface['Incident'] = surface['Incident'].replace({
    'Late Leaving Garage - Mechanical': 'Late Leaving Garage',
    'Late Leaving Garage - Operator': 'Late Leaving Garage',
    'Late Leaving Garage - Management': 'Late Leaving Garage'
})

surface = surface[(surface['Min Delay'] >= 0) & (surface['Min Delay'] <= 30)]

In [4]:
#Export merged data
surface.to_csv('./Resources/Cleaned/surface_master.csv')

In [5]:
surface.sort_values('Min Delay').head(15)

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,...,nDay,Max Temp (°C),Min Temp (°C),Mean Temp,Total Rain (mm),Total Snow (cm),Precipitation,Snow on Grnd (cm),Spd of Max Gust (km/h),Holiday
169091,2017-12-13,113,17.22,Monday,maint stn,Investigation,0.0,28.0,B/W,7717.0,...,13,-7.1,-14.0,-10.5,0.0,0.4,0.3,7.0,28,None
480080,2015-09-16,501,19.50,Monday,NaN,Investigation,0.0,0.0,W/B,4053.0,...,16,27.9,15.2,21.6,0.0,0.0,0.0,0.0,31,None
145081,2017-08-20,36,23.50,Friday,Duncanwoods,Diversion,0.0,7.0,w,9112.0,...,20,27.6,15.0,21.3,0.0,0.0,0.0,0.0,36,None
117661,2017-03-26,134,7.05,Friday,Centennial College Loop,Late Leaving Garage,0.0,7.0,W,1251.0,...,26,6.5,0.6,3.6,1.7,0.0,2.7,0.0,53,None
58407,2018-05-22,320,7.83,Sunday,Montgomery Ave and Yonge,Mechanical,0.0,0.0,N/B,8322.0,...,22,17.2,10.2,13.8,1.9,0.0,4.5,0.0,33,None
145084,2017-08-21,7,0.00,Saturday,Bathurst Route,Diversion,0.0,0.0,b/w,NaN,...,21,28.9,17.3,23.1,0.2,0.0,0.2,0.0,31,None
5862,2019-02-01,73,6.73,Wednesday,Royal York and LaRose,Late Leaving Garage,0.0,27.0,N/B,8053.0,...,1,-9.2,-18.7,-14.0,0.0,0.0,0.0,16.0,24,None
13740,2019-03-03,138,6.37,Friday,Sweels and Brenroy,Late Leaving Garage,0.0,5.0,N/B,1567.0,...,3,-1.4,-8.2,-4.8,0.0,0.2,0.2,11.4,25,None
117639,2017-03-26,124,5.97,Friday,Lawrence Stn,Mechanical,0.0,12.0,e,1684.0,...,26,6.5,0.6,3.6,1.7,0.0,2.7,0.0,53,None
439243,2018-09-14,510,18.10,Wednesday,King Street,Mechanical,0.0,0.0,N/B,4435.0,...,14,27.7,19.0,23.4,0.0,0.0,0.0,0.0,31,None


In [6]:
holidays.head()

,Date,Holiday
0,2012-01-02,New Year's Day
1,2012-02-20,Family Day
2,2012-04-06,Good Friday
3,2012-05-21,Victoria Day
4,2012-07-02,Canada Day


In [7]:
weather.head()

,Date,Max Temp (°C),Min Temp (°C),Mean Temp,Total Rain (mm),Total Snow (cm),Precipitation,Snow on Grnd (cm),Spd of Max Gust (km/h)
0,2014-01-01,-9.4,-14.9,-12.2,0.0,0.0,0.0,7.4,32
1,2014-01-02,-15.3,-19.7,-17.5,0.0,0.4,0.8,7.2,38
2,2014-01-03,-7.6,-23.6,-15.6,0.0,0.0,0.0,7.6,34
3,2014-01-04,-0.1,-9.6,-4.9,0.0,0.9,1.0,7.6,43
4,2014-01-05,0.8,-2.6,-1.0,1.4,6.9,12.9,8.8,31


In [8]:
surface.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471257 entries, 0 to 494168
Data columns (total 23 columns):
Date                      471257 non-null datetime64[ns]
Route                     471257 non-null category
Time                      471257 non-null float64
Day                       471257 non-null object
Location                  470377 non-null object
Incident                  470328 non-null object
Min Delay                 471257 non-null float64
Min Gap                   470735 non-null float64
Direction                 462226 non-null object
Vehicle                   412332 non-null category
Year                      471257 non-null int64
Type                      471257 non-null object
Month                     471257 non-null int64
nDay                      471257 non-null int64
Max Temp (°C)             471257 non-null float64
Min Temp (°C)             471257 non-null float64
Mean Temp                 471257 non-null float64
Total Rain (mm)           471257 non-null

<center><h2>Impact of Precipitation on Minutes Delayed by Route</center></h2>

In [9]:
routes = surface.groupby(['Date', 'Route']).agg({'Min Delay': 'sum', 'Precipitation': 'sum'}).reset_index()

routes_precip = routes[routes['Precipitation'] > 10]

routes_precip = routes_precip.sample(100)

### Histogram of Surface Routes - Minutes Delayed

In [ ]:
route_mean_min = routes_precip['Min Delay'].mean()
route_median_min = routes_precip['Min Delay'].median()
route_mean_precip = routes_precip['Precipitation'].mean()
route_median_precip = routes_precip['Precipitation'].median()

fig, axs = plt.subplots(nrows = 1, ncols = 2, sharey = True, figsize = [17, 5.5])

# Plot 1
ax = axs[0]
ax.hist(
    routes_precip['Min Delay'], 
    bins = 10,
    color = '#002147'
)

ax.axvline(
    route_mean_min, 
    lw = 2, 
    color = '#5F90B2',
    ls = '-.'
)

ax.axvline(
    route_median_min, 
    lw = 2, 
    color = 'red'
)

ax.set_ylabel('Count')
ax.set_xlabel('Minutes Delayed')

ax.text(-10, -17, f'Mean: {round(route_mean_min, 2)} \
                    \nMedian: {round(route_median_min, 2)}')
ax.grid(
    b = True, 
    which = 'major', 
    color = '#666666', 
    linestyle = '-'
)

ax.minorticks_on()
ax.grid(
    b = True, 
    which = 'minor', 
    color = '#666666', 
    linestyle = '-', 
    alpha = 0.2
)

# Plot 2
ax = axs[1]

ax.hist(
    routes_precip['Precipitation'], 
    bins = 10, 
    color = '#002147'
)

ax.axvline(
    route_mean_precip, 
    lw = 2, 
    color = '#5F90B2',
    ls = '-.'
)

ax.axvline(
    route_median_precip, 
    lw = 2, 
    color = 'red'
)

ax.set_ylabel('Count')
ax.set_xlabel('Precipitation (mm)')

ax.text(-10, -17, f'Mean: {round(route_mean_precip, 2)} \
                    \nMedian: {round(route_median_precip, 2)}')

ax.grid(
    b = True, 
    which = 'major', 
    color = '#666666', 
    linestyle = '-'
)

ax.minorticks_on()
ax.grid(
    b = True, 
    which = 'minor', 
    color = '#666666', 
    linestyle = '-', 
    alpha = 0.2
)

fig.suptitle('Grouped by Surface Routes Histograms of Minutes Delayed & Precipitation')
plt.text(-150, -30, 'Note: \n- Filtered for days with > 100 mm of precipitation.')

plt.savefig('TTC Surface Route Histograms (Min Delay, Precipitation).png')
plt.show()

### Linear Regression - Surface Routes: Precipitation vs. Minutes Delayed

In [ ]:
x = routes_precip['Precipitation']
y = routes_precip['Min Delay']

slope, intercept, rvalue, pvalue, std_err = stats.linregress(x, y)
fit = slope * x + intercept

In [ ]:
fig, ax = plt.subplots(1, figsize = [12.5, 7])

ax.scatter(
    routes_precip['Precipitation'], 
    routes_precip['Min Delay'],
    color = '#002147'
)

ax.plot(routes_precip['Precipitation'], fit, color = '#5F90B2', linewidth = 2.5)

ax.set_ylabel('Minutes Delayed')
ax.set_xlabel('Precipitation (Snow + Rain in mm)')
ax.set_title('By Surface Route # - Precipitation vs Minutes Delayed (5.3 Years)')

plt.grid(which = 'major', color = '#666666', linestyle = '-')
plt.minorticks_on()
plt.grid(which = 'minor', color = '#666666', linestyle = '-', alpha = 0.2)

plt.text(115, -150, f'Slope: {round(slope, 2)} \
                    \nIntercept: {round(intercept, 2)} \
                    \nR Value: {round(rvalue, 2)} \
                    \nR Squared: {round(rvalue**2, 2)} \
                    \nP Value: {round(pvalue, 2)} \
                    \nStandard Error: {round(std_err, 2)}')

plt.show()

<center><h2>Exploratory Scatterplot</center</h2>

In [ ]:
fig, ax = plt.subplots(1, figsize = [12.5, 7])

ax.scatter(
    surface['Precipitation'], 
    surface['Min Delay'], 
    c=surface['Mean Temp'], 
    alpha = 0.5
)

ax.set_ylabel('Minutes')
ax.set_xlabel('Daily Precipitation')
ax.set_title('Surface Routes - Temperature vs. Minutes Delayed')

plt.text(-30, -2000, 'Note: \
                    \n- Temperature in degrees celsius. \
                    \n- Outlier of 246245 (2014-03-03) minutes delayed removed. \
                    \n- Colored by Mean Temperature per Day')

plt.tight_layout()
plt.savefig('Surface Routes - Temperature vs. Minutes Delayed.png')
plt.show()

In [ ]:
vehicles = surface.groupby(['Date', 'Route', 'Vehicle']).agg({'Min Delay': 'sum', 'Min Gap': 'sum', 'Precipitation': 'sum'})
vehicles = vehicles.rename(columns={'Min Delay': 'Total Minutes', 'Min Gap': 'Total Gap'}).reset_index()
vehicles.head(10)

In [ ]:
vehicles['Total Minutes'].mean(), vehicles['Total Gap'].mean()

In [ ]:
vehicles = vehicles[vehicles['Total Minutes'] < 200]

fig, ax = plt.subplots(1, figsize=[12.5, 7])

ax.hist(vehicles['Total Minutes'], bins = 100)

plt.show()

<center><h2>Delay Time</h2></center>

In [ ]:
groupedbyday = surface.groupby(['Date', 'Day']).agg({'Min Delay': 'sum'})
groupedbyday = groupedbyday.reset_index()
groupedbyday

In [ ]:


fig, ax = plt.subplots(1, figsize = [12.5, 7])

ax.hist(groupedbyday['Min Delay'], bins = 100)

plt.show()

In [ ]:
groupedbyday.mean(), groupedbyday.median()

In [ ]:
totaldays = len(groupedbyday)
totalmin = groupedbyday['Min Delay'].sum()
meanminperday = round(groupedbyday['Min Delay'].mean(), 2)
medianminperday = round(groupedbyday['Min Delay'].median(), 2)
daysdelay = round((groupedbyday['Min Delay'].sum()/60)/24, 2)
percdelay = round((daysdelay / totaldays)/100, 2)
sigmaday = round(groupedbyday['Min Delay'].std(), 2)
delayvariance = round(sigmaday ** 2, 2)

delaystats_dict = {
    'Total Days': totaldays,
    'Total Minutes': daysdelay,
    'Mean Min. per Day': meanminperday,
    'Median Min. per Day': medianminperday,
    '% of Days Delayed': percdelay,
    'Daily Sigma': sigmaday,
    'Delay Variance': delayvariance
}

delaystats = pd.DataFrame(delaystats_dict, index=['Stats'])

delaystats = delaystats.style.format({
    '% of Days Delayed': '{:.2%}'.format
})

In [ ]:
totaldays, totalmin, meanminperday, medianminperday, daysdelay, percdelay, sigmaday, delayvariance

In [ ]:
delaystats

In [ ]:
surface.head()

In [ ]:
incidents = surface.groupby('Incident').agg({'Min Delay': 'sum', 'Min Gap': 'sum'}).sort_values('Min Delay')
incidents = incidents.reset_index()
incidents

In [ ]:
names = incidents['Incident'].unique().tolist()
names

In [ ]:
bar1 = weekdays['Min Delay']
bar2 = weekdays['Min Gap']

label = weekdays['Day']
barwidth = 1
names = weekdays['Day'].unique().tolist()
dayorder = [1, 5, 6, 4, 0, 2, 3]
names = [names[i] for i in dayorder]

In [ ]:
bar1 = incidents['Min Delay']
bar2 = incidents['Min Gap']

label = incidents['Incident']
barwidth = 1

description = incidents['Incident'].unique().tolist()
description_order = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
descriptions = [description[i] for i in description_order]

bars = np.add(bar1, bar2).tolist()

fig, ax = plt.subplots(1, figsize = [12.5, 10])

p1 = ax.bar(label, bar1, color='#800000', edgecolor='white', width=barwidth)
p2 = ax.bar(label, bar2, bottom=bar1, color='#888888', edgecolor='white', width=barwidth)

ax.set_ylabel('Minutes')
ax.set_xlabel('Days of the Week')
ax.set_title('Sum of Minutes per Delay Description')

plt.legend((p1[0], p2[0]), ('Minutes Delayed', 'Gap in Minutes'))

plt.xticks(label, descriptions, rotation = 35)

plt.tight_layout()

plt.savefig('Sum of Minutes per Delay Description.png')
plt.show()

<center><h2>Time of Day</center></h2>

In [ ]:
surface.head()

In [ ]:
surface.info()

In [ ]:
surface.head()

In [ ]:

dailyincidents.head()

In [ ]:
timeofday = surface.groupby(['Day', 'Time']).agg({'Min Delay': 'sum'}).reset_index()
timeofday = timeofday[timeofday['Time'] < 24]
timeofday['Hour'] = round(((timeofday['Time'] / 1) * 60) / 60, 0)
dailyincidents = surface.groupby(['Day', 'Time']).agg({'Min Delay': 'count'}).reset_index()
dailyincidents = dailyincidents.rename(columns={'Min Delay': 'Counts'})
timeofday['Counts'] = dailyincidents['Counts']

timeofday.head()

In [ ]:
weeklysum = timeofday.groupby(['Day', 'Hour']).agg({'Min Delay': 'sum', 'Counts': 'sum'}).reset_index().reset_index()
weeklysum = weeklysum.rename(columns={'index': 'Chronological', 'Min Delay': 'Min Sum'})
minmean = timeofday.groupby(['Day', 'Hour']).agg({'Min Delay': 'mean'}).rename(columns={'Min Delay': 'Min Mean'}).reset_index()
weeklysum['Min Mean'] = round(minmean['Min Mean'], 2)


weeklysum.tail()

In [ ]:
weeklysum.head()

In [ ]:
ticks = [12, 36, 65, 90, 115, 140, 165]
labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig, ax = plt.subplots(1, figsize = [12.5, 7])

ax.plot(
    weeklysum['Chronological'], 
    weeklysum['Min Mean'],
    c = '#5d0000',
    linewidth = 4
)

plt.xticks(ticks = ticks, labels = labels)


plt.show()

### Incidents

In [ ]:
surface.head()

In [ ]:
incidents = surface.groupby(['Incident']).agg({'Min Delay': 'sum', 'Precipitation': 'sum'}).reset_index()
incidents

In [ ]:
fig, ax = plt.subplots(1, figsize = [12.5, 7])

ax.bar(incidents['Incident'], incidents['Min Delay'])

plt.xticks(rotation=65)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize = [12.5, 7])

ax.scatter(incidents['Precipitation'], incidents['Min Delay'])

plt.show()